In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importando Bibliotecas

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from gensim import utils
from sys import stdout
import pandas as pd
import numpy as np
import unicodedata
import argparse
import gensim
import nltk
import re
import os

# Carregando dataset

In [13]:
path_dataset = '/content/drive/Shareddrives/Aprendizado por reforço/NLP/Dataset/dataset_projeto_final.csv'
df = pd.read_csv(path_dataset)

In [14]:
df['label'].value_counts()

Dano moral                                   9280
Inexistência de relação jurídica/débito      8843
Jurisprudência                               4863
Tutela antecipada                            4610
Inscrição indevida                           3790
Inversão do ônus da prova                    3649
Fraude                                       3444
Qualificação                                 2832
Justiça gratuita                             2718
Falha prestação/responsabilidade objetiva    2572
Relação de consumo                           2143
Produtos bancários                           1900
Dano material                                1707
Restituição em dobro                         1623
Responsabilidade civil                       1549
Canais Internos                              1468
Valor da causa                               1330
Produção de provas                           1132
Audiência conciliação                         843
Honorários/custas                             829


In [15]:
df

,text,label
0,Impossibilitado de arcar com as custas process...,Justiça gratuita
1,"Com efeito , faz jus o Autor à concessão da gr...",Justiça gratuita
2,"Como é cediço , de acordo com o referido diplo...",Justiça gratuita
3,"Por tais razões , e por imperiosa necessidade ...",Justiça gratuita
4,O Autor entabulou com o Banco Réu o Contrato d...,Financiamento veículo
...,...,...
64168,"Neste sentido , no que se refere à Tarifa de C...",Jurisprudência
64169,"Como também , tal entendimento foi pacificado ...",Jurisprudência
64170,“ A COBRANÇA A TÍTULO DE COMISSÃO DO CORRESPON...,Jurisprudência
64171,Assim já decidiu o STJ : RECURSO ESPECIAL REPE...,Jurisprudência


# Pré-processamento utilizado no corpus de treino do word2vec

In [16]:
# As classes 'Notificação de inscrição' e 'Inscrição indevida' são parecidas, então optamos por juntá-las em uma só
df['label'] = df['label'].replace(['Notificação de inscrição'], 'Inscrição indevida')

In [17]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [18]:
punctuations = re.escape('!"#%\'()*+,./:;<=>?@[\\]^_`{|}~')

re_remove_brackets = re.compile(r'\{.*\}')
re_remove_html = re.compile(r'<(\/|\\)?.+?>', re.UNICODE)
re_transform_numbers = re.compile(r'\d', re.UNICODE)
re_transform_emails = re.compile(r'[^\s]+@[^\s]+', re.UNICODE)
re_transform_url = re.compile(r'(http|https)://[^\s]+', re.UNICODE)
re_quotes_1 = re.compile(r"(?u)(^|\W)[‘’′`']", re.UNICODE)
re_quotes_2 = re.compile(r"(?u)[‘’`′'](\W|$)", re.UNICODE)
re_quotes_3 = re.compile(r'(?u)[‘’`′“”]', re.UNICODE)
re_dots = re.compile(r'(?<!\.)\.\.(?!\.)', re.UNICODE)
re_punctuation = re.compile(r'([,";:]){2},', re.UNICODE)
re_hiphen = re.compile(r' -(?=[^\W\d_])', re.UNICODE)
re_tree_dots = re.compile(u'…', re.UNICODE)
re_punkts = re.compile(r'(\w+)([%s])([ %s])' %
                       (punctuations, punctuations), re.UNICODE)
re_punkts_b = re.compile(r'([ %s])([%s])(\w+)' %
                         (punctuations, punctuations), re.UNICODE)
re_punkts_c = re.compile(r'(\w+)([%s])$' % (punctuations), re.UNICODE)
re_changehyphen = re.compile(u'–')
re_doublequotes_1 = re.compile(r'(\"\")')
re_doublequotes_2 = re.compile(r'(\'\')')
re_trim = re.compile(r' +', re.UNICODE)

def clean_text(text):
    """Apply all regex above to a given string."""
    text = text.lower()
    text = text.replace('\xa0', ' ')
    text = re_tree_dots.sub('...', text)
    text = re.sub('\.\.\.', '', text)
    text = re_remove_brackets.sub('', text)
    text = re_changehyphen.sub('-', text)
    text = re_remove_html.sub(' ', text)
    text = re_transform_numbers.sub('0', text)
    text = re_transform_url.sub('URL', text)
    text = re_transform_emails.sub('EMAIL', text)
    text = re_quotes_1.sub(r'\1"', text)
    text = re_quotes_2.sub(r'"\1', text)
    text = re_quotes_3.sub('"', text)
    text = re.sub('"', '', text)
    text = re_dots.sub('.', text)
    text = re_punctuation.sub(r'\1', text)
    text = re_hiphen.sub(' - ', text)
    text = re_punkts.sub(r'\1 \2 \3', text)
    text = re_punkts_b.sub(r'\1 \2 \3', text)
    text = re_punkts_c.sub(r'\1 \2', text)
    text = re_doublequotes_1.sub('\"', text)
    text = re_doublequotes_2.sub('\'', text)
    text = re_trim.sub(' ', text)
    return text.strip()

In [19]:
# Pre-processando todas as linhas
for index, row in df.iterrows():
    df['text'][index] = clean_text(row['text'])

In [20]:
possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

label_dict
df['label'] = df.label.replace(label_dict)

In [21]:
df

,text,label
0,impossibilitado de arcar com as custas process...,0
1,"com efeito , faz jus o autor à concessão da gr...",0
2,"como é cediço , de acordo com o referido diplo...",0
3,"por tais razões , e por imperiosa necessidade ...",0
4,o autor entabulou com o banco réu o contrato d...,1
...,...,...
64168,"neste sentido , no que se refere à tarifa de c...",14
64169,"como também , tal entendimento foi pacificado ...",14
64170,a cobrança a título de comissão do corresponde...,14
64171,assim já decidiu o stj : recurso especial repe...,14


# Carregando os embeddings pré-treinados do repositório nilc s300 em português utilizando a abordagem skipgram.

In [60]:
!wget -c http://143.107.183.175:22980/download.php?file=embeddings/word2vec/skip_s300.zip

--2023-02-05 14:42:20--  http://143.107.183.175:22980/download.php?file=embeddings/word2vec/skip_s300.zip
Connecting to 143.107.183.175:22980... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



In [ ]:
!unzip "/content/download.php?file=embeddings%2Fword2vec%2Fskip_s300.zip" -d "/content/Wod2vec/"

In [62]:
path_embeddings = '/content/Wod2vec/skip_s300.txt'

# Carregar um modelo pré-treinado de Word2Vec
model = KeyedVectors.load_word2vec_format(path_embeddings)

In [63]:
model.most_similar('amou', topn=5)

[('amava', 0.75396329164505),
 ('ama', 0.688590407371521),
 ('amei', 0.6054121255874634),
 ('amará', 0.604904055595398),
 ('odiou', 0.583797812461853)]

# Separando o dataset em treino e teste

In [64]:
X_train, X_test  = train_test_split(df , test_size=0.2, random_state=17, stratify = df['label'])


In [65]:
X_train_text = X_train['text']
X_test_text = X_test['text']

y_train_label = X_train['label']
y_test_label = X_test['label']

# Transformando o texto nos embeddings

In [66]:
# Converter um texto em um vetor de características

def vectorize_text(text):
    vector = [model.wv[word] for word in text.split() if word in model.wv]
    if len(vector) == 0:
        return np.zeros(model.vector_size)
    mean = sum(vector)/len(vector)
    if np.isnan(mean).any():
        return np.zeros(model.vector_size)
    return mean

# Converta uma lista de textos em uma lista de vetores de características
X_train = [vectorize_text(text) for text in X_train_text]
y_train = [label for label in y_train_label]


<ipython-input-66-9ec37de98969>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector = [model.wv[word] for word in text.split() if word in model.wv]


# Treinando o modelo

In [67]:
# Treine um classificador RandomForest usando os vetores de características
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [68]:
X_test = [vectorize_text(text) for text in X_test_text]
y_test = [label for label in y_test_label]

<ipython-input-66-9ec37de98969>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector = [model.wv[word] for word in text.split() if word in model.wv]


# Fazendo o teste

In [69]:
# Faça previsões com o modelo treinado
y_pred = classifier.predict(X_test)

# Obtenha o relatório de classificação
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.80      0.84       544
           1       0.90      0.16      0.27       112
           2       0.42      0.84      0.56      1769
           3       0.64      0.09      0.15       341
           4       0.82      0.51      0.63       325
           5       0.82      0.83      0.83       730
           6       0.69      0.27      0.39       294
           7       0.64      0.91      0.75      1856
           8       0.70      0.33      0.45       785
           9       0.70      0.62      0.66       922
          10       0.69      0.39      0.50       147
          11       0.60      0.26      0.36       380
          12       0.66      0.30      0.41       689
          13       0.94      0.32      0.48       156
          14       0.84      0.85      0.85       973
          15       0.96      0.99      0.97       566
          16       1.00      0.06      0.11        18
          17       0.78    